### 📦 Installing Required Packages

To ensure all dependencies for the project are installed, we use the following command:

In [ ]:
%pip install -r requirements.txt

### 🛠️ Import Required Modules and Classes

This block loads all essential modules and classes from the `sokegraph` package and its dependencies. 

In [1]:
from sokegraph.sources.base_paper_source import BasePaperSource
from sokegraph.sources.semantic_scholar_source import SemanticScholarPaperSource
from sokegraph.sources.pdf_paper_source import PDFPaperSource
from sokegraph.ranking.paper_ranker import PaperRanker
from sokegraph.graph.knowledge_graph import KnowledgeGraph
from sokegraph.util.logger import LOG
from sokegraph.agents.ai_agent import AIAgent
from sokegraph.agents.openai_agent import OpenAIAgent
from sokegraph.agents.gemini_agent import GeminiAgent
from sokegraph.ontology.ontology_updater import OntologyUpdater
from sokegraph.graph.neo4j_knowledge_graph import Neo4jKnowledgeGraph
from sokegraph.agents.llama_agent import LlamaAgent
from sokegraph.agents.ollama_agent import OllamaAgent
from sokegraph.agents.claude_agent import ClaudeAgent
from sokegraph.sources.journal_api_source import JournalApiPaperSource
from sokegraph.graph.networkx_knowledge_graph import NetworkXKnowledgeGraph
import json

2025-09-10 18:24:33,453 [INFO ]	sokegraph v1.1
2025-09-10 18:24:33,453 [INFO ]	ipykernel_launcher.py --f=/Users/shahlla/Library/Jupyter/runtime/kernel-v3106818bbd8e1965fde9052ff855fde6b9b184404.json


# ✨ User Input Form

This step initializes the **interactive interface** for entering parameters.  

➡️ First, you will be prompted to **select the paper source**.  
➡️ After that, the form will expand to show the **remaining input fields** needed for the pipeline.  

> 💡 Use this form to configure all inputs before running the pipeline.


In [2]:
import importlib
import sokegraph.ui.ui_inputs
importlib.reload(sokegraph.ui.ui_inputs)
from sokegraph.ui.ui_inputs import SOKEGraphUI

# Create UI instance
ui = SOKEGraphUI()

# Display the UI in the notebook
ui.display_ui()


## 🧠 Step 0: Select and Initialize AI Agent

- The pipeline selects the appropriate AI agent based on your UI choice (`ui.params.AI`).
- Supported providers: `openAI`, `gemini`, `llama`, `ollama`, `claude`.
- For all except `ollama`, an API key file is required.

**How it works:**
- The code checks your selection and creates the corresponding agent instance.
- If an unsupported provider is chosen, it raises an error.

> **Tip:**  
> Make sure your API key file exists and is valid for the selected provider (except `ollama`).


In [3]:
LOG.info("🚀 Starting Full Pipeline")

# 0. Setup AI agent
ai_tool: AIAgent
if ui.params.AI == "openAI":
    ai_tool = OpenAIAgent(ui.params.api_keys_file, ui.params.field_of_interest)
elif ui.params.AI == "gemini":
    ai_tool = GeminiAgent(ui.params.api_keys_file, ui.params.field_of_interest)
elif ui.params.AI == "llama":
    ai_tool = LlamaAgent(ui.params.api_keys_file, ui.params.field_of_interest)
elif ui.params.AI == "ollama":
    ai_tool = OllamaAgent(ui.params.field_of_interest)
elif ui.params.AI == "claude":
    ai_tool = ClaudeAgent(ui.params.api_keys_file, ui.params.field_of_interest)
else:
    raise ValueError(f"Unsupported AI provider: {ui.params.AI}")

2025-09-10 18:25:17,490 [INFO ]	🚀 Starting Full Pipeline
2025-09-10 18:25:17,542 [INFO ]	Using MPS GPU for AI agent.


## 📄 Step 1: Select and Fetch Papers

- Chooses the paper source based on your UI selection:
  - **Semantic Scholar:** Needs `number_papers` and `paper_query_file`
  - **PDF Zip:** Needs `pdfs_file`
  - **Journal API:** Needs `paper_query_file` and `api_key_file`
- Logs an error if required inputs are missing or the source is invalid.
- Calls `fetch_papers()` to get the papers for the next steps.

> ⚠️ Make sure you provide the correct files and parameters for your chosen source.

In [5]:
# 1. Select paper source
paper_source: BasePaperSource

if ui.params.paper_source == "Semantic Scholar":
    if not ui.params.number_papers or not ui.params.paper_query_file:
        LOG.error("❌ 'number_papers' and 'paper_query_file' are required for Semantic Scholar source.")
    else:
        paper_source = SemanticScholarPaperSource(
            num_papers=int(ui.params.number_papers),
            query_file=ui.params.paper_query_file,
            output_dir=ui.params.output_dir
        )

elif ui.params.paper_source == "PDF Zip":
    if not ui.params.pdfs_file:
        LOG.error("❌ 'pdfs_file' (ZIP file) is required for PDF source.")
    else:
        paper_source = PDFPaperSource(
            zip_path=ui.params.pdfs_file,
            output_dir=ui.params.output_dir
        )

elif ui.params.paper_source == "Journal API":
    if not ui.params.paper_query_file or not ui.params.api_key_file:
        LOG.error("❌ 'paper_query_file' and 'api_key_file' are required for Journal API source.")
    else:
        paper_source = JournalApiPaperSource(
            query_file=ui.params.paper_query_file,
            api_key_file=ui.params.api_key_file,
            output_dir=ui.params.output_dir
        )

else:
    LOG.error("❌ Invalid or unsupported paper source selected.")
    paper_source = None

# 2. Fetch papers
if paper_source:
    papers_path = paper_source.fetch_papers()
else:
    papers_path = ""


2025-09-10 18:28:47,588 [INFO ]	Searching Semantic Scholar for: earth-abundant catalysts for water electrolysis
INFO:sokegraph:Searching Semantic Scholar for: earth-abundant catalysts for water electrolysis


## 🧠 Step 2: Update Ontology

- Uses the retrieved papers and selected AI agent to enrich the ontology file.
- The updated ontology is saved for the next steps.

In [6]:
import importlib
import sokegraph.agents.openai_agent
importlib.reload(sokegraph.agents.openai_agent)
from sokegraph.agents.openai_agent import OpenAIAgent
import sokegraph.agents.ai_agent
importlib.reload(sokegraph.agents.ai_agent)
from sokegraph.agents.ai_agent import AIAgent
import sokegraph.ontology.ontology_updater
importlib.reload(sokegraph.ontology.ontology_updater)
from sokegraph.ontology.ontology_updater import OntologyUpdater
# 2. Update ontology
ontology_updater = OntologyUpdater(ui.params.ontology_file, papers_path, ai_tool, ui.params.output_dir)  # or however you instantiate it
updated_ontology_path = ontology_updater.enrich_with_papers()

2025-09-10 18:28:53,899 [INFO ]	🔍 Processing paper: 4b32cbadd7ecebe2e56366f5c91fadb2a3d9af31
INFO:sokegraph.agents.ai_agent:🔍 Processing paper: 4b32cbadd7ecebe2e56366f5c91fadb2a3d9af31


papers :                                    paper_id  \
0   4b32cbadd7ecebe2e56366f5c91fadb2a3d9af31   
1   68aff3d2512db636e96abb121aecb54f7c0f9dc0   
2   0ca67c3e9dfe0122b82d41ad6aef22338f924fc0   
3   fda4e98ca331d5372b752d16469789d2603a216f   
4   9df40f454e228f5be82dce31e1d0b3ba7695881f   
5   0cf6cb6f34a4a010066d6fe63a25ce68632d8a8d   
6   fef649db95eda5370120b5cff5b30c5e8a6174d9   
7   ce495ca4313f3425b5f6cc3c0df79ee517d79a6e   
8   89f932ef82520c20be8cc9f42c3522854f6ae1bf   
9   3e8fdf5af0d87bfc8e8761ed8af3db47f7484ac8   
10  43fa3ca34a030051b26fb64ae93a4e70345d2619   
11  5fc1ba02dd80ad6d3182018e2e6f1917ca98445d   
12  4761ec4d8af507e6748c1a1da895c153475e2287   
13  cbf2ecee0a6a6e33d3c601940d8c5e3660505998   
14  cb3d479bf95f391432ceec124ba28cf68ee4ae29   
15  21319ce82dcc38dcecb4901b0501385a1b0e6ab4   
16  3972080e97c640c1281d30643f122248a138e71c   
17  ebe5c4b2ea2bb590f7a7296c61353f3065dedc52   
18  fa3a34bef23a155556e99d1be6ce7ae44a7a107d   
19  93730a945c9911f515429c07e4dc

2025-09-10 18:28:55,984 [WARNI]	⚠️ No items extracted for paper 4b32cbadd7ecebe2e56366f5c91fadb2a3d9af31, layer 'Environment'


model_output : ```json
[]
```
⚠️ JSON parsing failed. Attempting fallback regex... pattern: "layer":\s*"([^"]+)",\s*"matched_category":\s*"([^"]+)",\s*"keyword":\s*"([^"]+)",\s*"meta_data":\s*"([^"]+)"
pattern : "layer":\s*"([^"]+)",\s*"matched_category":\s*"([^"]+)",\s*"keyword":\s*"([^"]+)",\s*"meta_data":\s*"([^"]+)"
matches : []
✅ Extracted {} items via regex.
results : {} for paper_id : 4b32cbadd7ecebe2e56366f5c91fadb2a3d9af31
xxxx


2025-09-10 18:28:57,242 [WARNI]	⚠️ No items extracted for paper 4b32cbadd7ecebe2e56366f5c91fadb2a3d9af31, layer 'Process'


...

2025-09-10 18:29:53,885 [ERROR]	❌ An unexpected system error occurred: {str(e)}
ERROR:sokegraph.agents.openai_agent:❌ An unexpected system error occurred: {str(e)}
2025-09-10 18:29:53,887 [WARNI]	⚠️ No items extracted for paper 0cf6cb6f34a4a010066d6fe63a25ce68632d8a8d, layer 'Application'
2025-09-10 18:29:53,889 [INFO ]	🔍 Processing paper: fef649db95eda5370120b5cff5b30c5e8a6174d9
INFO:sokegraph.agents.ai_agent:🔍 Processing paper: fef649db95eda5370120b5cff5b30c5e8a6174d9


model_output : ```json
[
    {
        "layer": "Application",
        "matched_category": "Renewable Energy Integration",
        "keyword": "photoelectrocatalysis",
        "meta_data": "The different design principles of the OER catalysts in photoelectrocatalysis and photocatalysis systems are also presented."
    },
    {
        "layer": "Application",
        "matched_category": "Renewable Energy Integration",
        "keyword": "photocatalysis",
        "meta_data": "The different design principles of the OER catalysts in photoelectrocatalysis and photocatalysis systems are also presented."
    },
    {
        "layer": "Application",
        "matched_category": "Renewable Energy Integration",
        "keyword": "PV-E",
        "meta_data": "Finally, the future research directions of electrocatalysts for water splitting, and coupling of photovoltaic (PV) panel with a water electrolyzer, so called PV-E, are given as perspectives."
    }
]
```
⚠️ JSON parsing failed. Attempting fa

2025-09-10 18:29:54,900 [WARNI]	⚠️ No items extracted for paper fef649db95eda5370120b5cff5b30c5e8a6174d9, layer 'Environment'


model_output : ```json
[]
```
⚠️ JSON parsing failed. Attempting fallback regex... pattern: "layer":\s*"([^"]+)",\s*"matched_category":\s*"([^"]+)",\s*"keyword":\s*"([^"]+)",\s*"meta_data":\s*"([^"]+)"
pattern : "layer":\s*"([^"]+)",\s*"matched_category":\s*"([^"]+)",\s*"keyword":\s*"([^"]+)",\s*"meta_data":\s*"([^"]+)"
matches : []
✅ Extracted {} items via regex.
results : {} for paper_id : fef649db95eda5370120b5cff5b30c5e8a6174d9
xxxx


KeyboardInterrupt: 

## 🕸 Step 3: Build the Knowledge Graph

- Builds a knowledge graph from the enriched ontology.
- Supports two backends:
  - **Neo4j:** Uses credentials from a JSON file.
  - **NetworkX:** Builds an in-memory graph.
- The graph is created and ready for exploration or analysis.

In [ ]:
import importlib
import sokegraph.graph.knowledge_graph
import sokegraph.graph.networkx_knowledge_graph
importlib.reload(sokegraph.graph.knowledge_graph)
importlib.reload(sokegraph.graph.networkx_knowledge_graph)
from sokegraph.graph.networkx_knowledge_graph import NetworkXKnowledgeGraph

# 3. Build knowledge graph
LOG.info(" Building knowledge graph ....")


#### build graph
graph_builder: KnowledgeGraph
if(ui.params.kg_type == "neo4j"):
    ### load
    with open(ui.params.kg_credentials_file, "r") as f:
        credentials = json.load(f)
    graph_builder = Neo4jKnowledgeGraph(ontology_updater.output_path, 
                                        credentials["neo4j_uri"],
                                        credentials["neo4j_user"],
                                        credentials["neo4j_pass"])
elif(ui.params.kg_type == "networkx"):
    graph_builder = NetworkXKnowledgeGraph(ontology_updater.output_path)

graph_builder.build_graph()


LOG.info("🎉 Pipeline Completed Successfully")

2025-09-03 13:30:35,684 [INFO ]	 Building knowledge graph ....
INFO:sokegraph: Building knowledge graph ....
2025-09-03 13:30:35,692 [INFO ]	🎉 Pipeline Completed Successfully
INFO:sokegraph:🎉 Pipeline Completed Successfully


meta : {'value': 0.5, 'unit': 'M'}
meta : {'value': 2.0, 'unit': 'SO'}
meta : {'value': 4.0, 'unit': 'combined'}
meta : {'value': 0.5, 'unit': 'M'}
meta : {'value': 2.0, 'unit': 'SO'}
meta : {'value': 4.0, 'unit': 'combined'}
meta : {'value': 0.5, 'unit': 'M'}
meta : {'value': 2.0, 'unit': 'SO'}
meta : {'value': 4.0, 'unit': 'combined'}
meta : {'value': 0.5, 'unit': 'M'}
meta : {'value': 2.0, 'unit': 'SO'}
meta : {'value': 4.0, 'unit': 'combined'}
meta : {'value': 0.5, 'unit': 'M'}
meta : {'value': 2.0, 'unit': 'SO'}
meta : {'value': 4.0, 'unit': 'combined'}
meta : {'value': 0.5, 'unit': 'M'}
meta : {'value': 2.0, 'unit': 'SO'}
meta : {'value': 4.0, 'unit': 'combined'}
meta : {'value': 0.5, 'unit': 'M'}
meta : {'value': 2.0, 'unit': 'SO'}
meta : {'value': 4.0, 'unit': 'combined'}
meta : {'value': 0.5, 'unit': 'M'}
meta : {'value': 2.0, 'unit': 'SO'}
meta : {'value': 12.3, 'unit': 'mV'}
meta : {'value': 52.0, 'unit': 'mV'}
meta : {'value': 4.7, 'unit': 's-'}
meta : {'value': 0.5, 'unit'

In [1]:
from sokegraph.graph.networkx_knowledge_graph import NetworkXKnowledgeGraph
from sokegraph.graph.knowledge_graph import KnowledgeGraph
onotology_file = "external/output/updated_ontology.json"
graph_builder: KnowledgeGraph
graph_builder = NetworkXKnowledgeGraph(onotology_file)

graph_builder.build_graph()



Processing layer: Environment with categories: {'Acidic': [{'keywords': ['acidic environment', 'pH 1', 'pH<', 'acidic', 'pH<7', 'pH < 7', 'pH=1'], 'meta_data': 'For proton exchange membrane electrolyzers featuring an acidic environment, platinum group metals are currently used.', 'paper_id': '70e072fac8c65a84f3d614f3d48a49846ed27ba4', 'parsed_meta': []}, {'keywords': ['pH 1', 'acid', 'pH<', 'acidic', 'pH<7', 'pH < 7', 'pH=1'], 'meta_data': 'However, several earth abundant catalysts have been identified for HER in acid.', 'paper_id': '70e072fac8c65a84f3d614f3d48a49846ed27ba4', 'parsed_meta': []}, {'keywords': ['pH 1', 'pH<', 'acidic', 'pH<7', 'pH < 7', 'pH=1', '0.5 M H2SO4'], 'meta_data': 'Electrochemical measurements in 0.5 M H2SO4 combined with ex situ and quasi in situ XPS revealed a-MoSx films to transform into highly active a-MoS2.', 'paper_id': '70e072fac8c65a84f3d614f3d48a49846ed27ba4', 'parsed_meta': [{'value': 0.5, 'unit': 'M'}, {'value': 2.0, 'unit': 'SO'}, {'value': 4.0, 'uni

In [2]:

graph_builder.show_subgraph_interactive()  # opens the dropdown UI in Jupyter

## 📊 Step 4: Rank Papers Based on Ontology and Keywords

- Instantiates `PaperRanker` with the AI tool, fetched papers, enriched ontology, keywords file, and output directory.
- Calls `rank_papers()` to score and rank papers by relevance to your research interests.

In [ ]:
# 4. Rank papers
#LOG.info("ranking papers ....")
import importlib
import sokegraph.ranking.paper_ranker
importlib.reload(sokegraph.ranking.paper_ranker)
from sokegraph.ranking.paper_ranker import PaperRanker
ranker = PaperRanker(ai_tool, papers, ontology_updater.output_path, ui.params.keywords_file, ui.params.output_dir)
results_csv, results_all = ranker.rank_papers()